<h3>PreProcess: Update the existing USFWS Survey Plotting Data (.csv) file</h3>

Updates the very important data file: USFWS_Survey_Plotting_Data.csv. Use this notebook to add new survey data, as surveys are collected. 

**NOTE: The Plot Group 5 series relies on the USFWS_Survey_Plotting_Data.csv file to generate its box plots.**

<br />
**Instructions:**
1. Update the User Inputs cell, below, adding in the name of the new survey data spreadsheet and the id for the new survey
2. Under Cell -> Run All


**Changes:**
1. 3/31/2017 - Merged the phi 4 and remainder values into a single remainder column in the file USFWS_Survey_Plotting_Data.csv. The phi 4 screen was dropped early in the program and is no longer in use.

<h4>Import requisite modules and libraries</h4>

In [3]:
import numpy as np
import pandas as pd

<h4>User Inputs:</h4>

In [17]:
####################################################################################
datapath='/Users/paulp/GoogleDrive/projects/PeaIslandBeachMonitoring/data/' # where your existing Survey file is stored...
sspath='/Users/paulp/GoogleDrive/projects/PeaIslandBeachMonitoring/data/xls/' # where your spreadsheet is stored

spdfn='USFWS_Survey_Plotting_Data.csv'     # NOTE this file contains all surveys from 7/2014 thru ???, inclusive

nsfn ='FWSGrainSizeAnalysis2017_04.xlsx'   # name of new survey file whose data are to be added to spdfn
new_survey='201704'  
####################################################################################

<h4>Load data from the existing USFWS Survey Data file and from the new survey spreadsheet</h4>

In [18]:
cols=(['group','transect','sample','survey','phi_-1','phi_-0.5','phi_0','phi_0.5','phi_1','phi_1.25','phi_1.5','phi_1.75','phi_2','phi_2.5','phi_3','phi_3.5','remainder'])
spd_df=pd.read_csv(datapath+spdfn, names=cols)

ss_df=pd.read_excel(sspath+nsfn, skiprows=0)

In [6]:
ss_df

,sheet_code,sample_date,transect_id,sample_number,pan_weight,pan+wet_weight,pan+dry_weight,dry_weight,phi_-1,"phi_-0,5",...,% COARSE SAND:,% MEDIUM SAND:,% FINE SAND:,% V FINE SAND:,% V COARSE SILT:,% COARSE SILT:,% MEDIUM SILT:,% FINE SILT:,% V FINE SILT:,% CLAY:
0,apr2017_C1,2017-04-27,C1,S1,6.19,147.11,119.98,113.79,10.69,11.04,...,0,0,0,0,0,0,0,0,0,0
1,apr2017_C1,2017-04-27,C1,S2,5.82,114.18,97.20,91.38,3.25,3.93,...,0,0,0,0,0,0,0,0,0,0
2,apr2017_C1,2017-04-27,C1,S3,5.96,112.93,93.15,87.19,0.21,0.22,...,0,0,0,0,0,0,0,0,0,0
3,apr2017_C1,2017-04-27,C1,S4,5.67,116.60,116.58,110.91,0.00,0.02,...,0,0,0,0,0,0,0,0,0,0
4,apr2017_C1,2017-04-27,C1,S5,6.18,128.91,128.46,122.28,2.97,1.42,...,0,0,0,0,0,0,0,0,0,0
5,apr2017_C2,2017-04-27,C2,S1,6.17,167.02,149.53,143.36,88.73,16.04,...,0,0,0,0,0,0,0,0,0,0
6,apr2017_C2,2017-04-27,C2,S2,6.08,126.21,102.54,96.46,1.00,1.98,...,0,0,0,0,0,0,0,0,0,0
7,apr2017_C2,2017-04-27,C2,S3,6.16,114.20,102.52,96.36,0.00,0.00,...,0,0,0,0,0,0,0,0,0,0
8,apr2017_C2,2017-04-27,C2,S4,6.04,121.05,119.53,113.49,0.06,0.02,...,0,0,0,0,0,0,0,0,0,0
9,apr2017_C2,2017-04-27,C2,S5,6.03,124.06,122.53,116.50,1.23,3.93,...,0,0,0,0,0,0,0,0,0,0


<h4>Data Preprocessing I: (Converting the absolute screen weights from the spreadsheet into fraction weight percentages:</h4>

In [19]:
### Convert screen absolute weights posted in the spreadsheet df ss_df to weight percentages:

# recompute the total weight of the samples, just in case the summed_weight field is flawed...
ss_df['total_weight']=ss_df[['phi_-1', 'phi_-0,5','phi_0', 'phi_0,5','phi_1', 'phi_1,25','phi_1,5','phi_1,75',
'phi_2','phi_2,5','phi_3','phi_3,5','remainder']].sum(axis=1)

# recast the absolute screen weights to weight percent fractions: (This is meathead. There must be a better way!)
ss_df['phi_-1'] = (ss_df['phi_-1'] / ss_df['total_weight'])*100
ss_df['phi_-0,5'] = (ss_df['phi_-0,5'] / ss_df['total_weight'])*100
ss_df['phi_0'] = (ss_df['phi_0'] / ss_df['total_weight'])*100
ss_df['phi_0,5'] = (ss_df['phi_0,5'] / ss_df['total_weight'])*100
ss_df['phi_1'] = (ss_df['phi_1'] / ss_df['total_weight'])*100
ss_df['phi_1,25'] = (ss_df['phi_1,25'] / ss_df['total_weight'])*100
ss_df['phi_1,5'] = (ss_df['phi_1,5'] / ss_df['total_weight'])*100
ss_df['phi_1,75'] = (ss_df['phi_1,75'] / ss_df['total_weight'])*100
ss_df['phi_2'] = (ss_df['phi_2'] / ss_df['total_weight'])*100
ss_df['phi_2,5'] = (ss_df['phi_2,5'] / ss_df['total_weight'])*100
ss_df['phi_3'] = (ss_df['phi_3'] / ss_df['total_weight'])*100
ss_df['phi_3,5'] = (ss_df['phi_3,5'] / ss_df['total_weight'])*100
ss_df['remainder'] = (ss_df['remainder'] / ss_df['total_weight'])*100

<h4>Data Preprocessing II: (Adding the group field - classifies record as belonging either to the control or treatment group)

In [20]:
## create a populate new group column (field) based on transect id type (e.g., control or transect):
ss_df.loc[ss_df['transect_id'].str[0] == 'C', 'Group'] ='control'
ss_df.loc[ss_df['transect_id'].str[0] =='T', 'Group'] = 'treatment'

<h4>Data Preprocessing III: (Adding the survey, and 'dummy' phi_4, fields to the ss_df dataframe. We add the dummy phi 4 field to account for its existence in early surveys, and thus a presence in the Survey Data File. It's no longer used, but it's influence from the early days remains with us...)</h4>

In [21]:
ss_df['survey']=new_survey
ss_df['phi_4']=np.nan

<h4>Data Preprocessing IV: (re-sculpting the ss_df dataframe to match that of the spd_df. Actually, we'll create a new spreadsheet with just the stuff from ss_df that we want, in the order that we want, so that we can then proceed to merge (stack vertically) the two (spd_df + ss_df(new))

In [22]:
cols=(['group','transect','sample','survey','phi_-1','phi_-0.5','phi_0','phi_0.5','phi_1','phi_1.25','phi_1.5','phi_1.75','phi_2','phi_2.5','phi_3','phi_3.5','remainder'])

ss2_df=ss_df[['Group','transect_id','sample_number','survey','phi_-1','phi_-0,5','phi_0','phi_0,5','phi_1','phi_1,25','phi_1,5','phi_1,75','phi_2','phi_2,5','phi_3','phi_3,5','remainder']]
ss2_df.columns=cols

<h4>Merge thee two data frames, spd_df and ss2_df, to create one big one that will be written out, in the next cell, to the filesystem to create the latest iteration of the USFWS_Survey_Plotting_Data.csv file...</h4>

In [23]:
cat_df = pd.concat([spd_df, ss2_df], ignore_index=True)
cat_df=cat_df[cat_df.group.notnull()]

In [24]:
cat_df.dtypes

group         object
transect      object
sample        object
survey        object
phi_-1       float64
phi_-0.5     float64
phi_0        float64
phi_0.5      float64
phi_1        float64
phi_1.25     float64
phi_1.5      float64
phi_1.75     float64
phi_2        float64
phi_2.5      float64
phi_3        float64
phi_3.5      float64
remainder    float64
dtype: object

<h4>Write the cat_df dataframe out to external comma-separated values file, to replace the existing USFWS_Survey_Plotting_Data_file.csv:</h4>

In [25]:
cat_df.to_csv(datapath+'USFWS_Survey_Plotting_Data.csv', header=False, na_rep=0, float_format="%.3f", index=False)

<h4>The End</h4>   USFWS_Survey_Plotting_Data